In [ ]:
import os
import json
import xmltodict
from archiObjects import *

In [ ]:
# Read ARIS AML file & convert it to a dictionary
PATH = 'input'
AMLFILE = 'AppDepPat.xml'

aris_file = os.path.join(PATH, AMLFILE)


In [ ]:
class AML:

    def __init__(self, aml_file: str):
        self.data = xmltodict.parse(open(aml_file, 'r').read())
        self.folders = []
        self.model = OpenExchange('aris').OEF

    def parse_folders(self, grp = None) -> bool:
        if grp is None:
            grp = self.data

        if 'Group' not in grp:
            return False

        self.folders.append(grp['Group']['AttrDef']['AttrValue']['StyledElement'][1]['PlainText']['@TextValue'])
        self.parse_folders(grp)

        return True




In [ ]:
aris = AML(aris_file)


# Creation of an Archimate Open Exchange File (OEF)

1- Import libraries



2- Import OEF class library

In [3]:
# create new model
model = OpenExchange("My Model")


In [4]:
# Add elements
e1 = Element("IT Solution A", "ApplicationCollaboration")
e2 = Element("IT Solution B", "ApplicationCollaboration")
e3 = Element("MyApp", 'ApplicationComponent')


In [5]:
# Add relationship
r = Relationship(e1.uuid, e2.uuid, "Serving", name="serves")
model.add_relationship(r)

In [6]:
# Add properties - should be before Views
pdef = PropertyDefinitions()
pdef.add('Asset Owner')
model.add_property_def(pdef)
p1 = Property('Asset Owner', 'Frank Vanhulle', pdef).property
p2 = Property('Architect', 'Xavier Mayeur', pdef).property
e1.add_property(p1, p2)
model.add_element(e1, e2, e3)

In [7]:
# Add a view & a diagram
v = View('New_View')
model.add_view(v)

# Add nodes & connections to the view
gr = Element("Group", "Grouping")
ngr = Node(gr, 0, 0, 200, 150)
model.add_element(gr)

grel = Relationship(gr, e1, 'Composition')
model.add_relationship(grel)


n1 = Node(e1.uuid, 50, 50)
n2 = Node(e2.uuid, 300, 300, 400, 200)

c = Connection(r, n1, n2)
cgr = Connection(grel, ngr, n1)

v.add_node(ngr, n1, n2)
v.add_connection(c, cgr)

# add bendpoints
c.add_bendpoint((10+n1.w/2, 100+n1.h/2), (300+n2.w/2, 100+n2.h/2))

In [8]:
# Add nodes in node
n3 = Node(e3.uuid, 310, 310)
n2.add_node(n3)
r2 = Relationship(e2, e3, 'Aggregation')
c2 = Connection(r2, n2, n3)
model.add_relationship(r2)
v.add_connection(c2)

e4 = Element("MyData", 'DataObject')
model.add_element(e4)
r4 = Relationship(e3, e4, 'Access', accessType='Write')
model.add_relationship(r4)
n4 = Node(e4, 310, 400)
n2.add_node(n4)
c4 = Connection(r4, n3, n4)
v.add_connection(c4)

TypeError: __init__() got an unexpected keyword argument 'accessType'

In [ ]:
# Add a style
fc = RGBA()
lc = RGBA()
f = Font()
ff = RGBA()

fc.r = '255'
fc.g = '0'
fc.b = '0'
fc.a = '50'

lc.r = '0'
lc.g = '0'
lc.b = '255'
f.name = 'Segoe UI'
f.size = '10'
ff.r = '0'
ff.g = '0'
ff.b = '0'
f.color = ff

s = Style(fc, lc, f)
n4.add_style(s)

In [ ]:
print(json.dumps(model.OEF, indent=4))


In [ ]:
archi = model.OEF
file = os.path.join('output', 'out.json')
json.dump(archi, open(file, 'w'), indent=4)
file = os.path.join('output', 'out.xml')
xmltodict.unparse(archi, open(file, 'w'), pretty=True)

#out = xmltodict.unparse((archi))


In [ ]:
file = os.path.join('template', 'x.xml')
tpl = open(file, 'r').read()
archi = xmltodict.parse(tpl)
print(json.dumps(archi, indent=4))